# Dimension Modeling

In [0]:
%sql
select count(*) from parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`

count(1)
10100


# Implementing SCD Type 2 (Add New Row): 
**Maintains full history by adding new rows for changes, using start/end dates and active flags.**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType, LongType
from pyspark.sql.window import Window

# Get source data
df_src = spark.sql('''
    SELECT DISTINCT category_key, category_name
    FROM parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`
''')

# Initialize target DataFrame
if spark.catalog.tableExists('contoso_catalog.gold.dim_category'):
    df_tgt = spark.sql('''
        SELECT  category_id, 
               category_key, 
               category_name,
                is_current,
                start_date,
                end_date
        FROM contoso_catalog.gold.dim_category
    ''')
else:
    df_tgt = df_src.withColumn("category_id", F.lit(0).cast(LongType()))\
                   .withColumn("is_current", F.lit(1).cast(IntegerType()))\
                   .withColumn("start_date", F.current_date())\
                   .withColumn("end_date", F.lit(None).cast(DateType()))\
                   .filter("1 = 0")  # Empty Schema

def generate_surrogate_key(df, start_value):
    w = Window.orderBy("category_key")
    return df.withColumn("category_id", F.row_number().over(w) + F.lit(start_value))

def apply_scd_type2_changes(df_src, df_tgt):
    # Define the final column schema for consistency
    final_columns = [
        "category_id", 
        "category_key", 
        "category_name",
        "is_current",
        "start_date",
        "end_date"
    ]

    # Step 1: Handle empty target
    if df_tgt.rdd.isEmpty():
        print("Target DataFrame is empty. Initializing with source records.")
        initial_df = df_src.withColumn("category_id", F.lit(0).cast(LongType()))\
                          .withColumn("start_date", F.current_date())\
                          .withColumn("end_date", F.lit(None).cast(DateType()))\
                          .withColumn("is_current", F.lit(1).cast(IntegerType()))
        return generate_surrogate_key(initial_df, 0).select(final_columns)

    # Step 2: Get max surrogate key
    max_surrogate_key = df_tgt.agg(F.max("category_id")).collect()[0][0] or 0
    
    # Step 3: Join source and target
    src = df_src.alias("src")
    tgt = df_tgt.filter(F.col("is_current") == 1).alias("tgt")
    joined_df = src.join(tgt, "category_key", "outer")

    # Step 4: Identify new records
    new_records = joined_df.filter(F.col("tgt.category_id").isNull())\
                          .select("src.*")
    new_records_count = new_records.count()

    if new_records_count > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))\
                     .select(final_columns)
    else:
        new_records = spark.createDataFrame([], df_tgt.schema)

    # Step 5: Identify changed records
    changed_records = joined_df.filter(
        (F.col("tgt.category_id").isNotNull()) &
        (F.coalesce(F.col("src.category_name") != F.col("tgt.category_name"), F.lit(False)))
    )
    changed_records_count = changed_records.count()

    if changed_records_count > 0:
        # Step 5.1: Create new versions for changed records
        new_versions = changed_records.select("src.*")\
                                    .withColumn("start_date", F.current_date())\
                                    .withColumn("end_date", F.lit(None).cast(DateType()))\
                                    .withColumn("is_current", F.lit(1).cast(IntegerType()))
        start_key = max_surrogate_key + new_records_count + 1
        new_versions = generate_surrogate_key(new_versions, start_key).select(final_columns)

        # Step 5.2: Close old versions
        old_versions = df_tgt.join(
            changed_records.select("category_key"), "category_key", "inner"
        ).withColumn("end_date", F.when(
            F.col("is_current") == 1, F.date_sub(F.current_date(), 1)
        ).otherwise(F.col("end_date")))\
         .withColumn("is_current", F.lit(0))\
         .select(final_columns)
    else:
        new_versions = spark.createDataFrame([], df_tgt.schema)
        old_versions = spark.createDataFrame([], df_tgt.schema)

    # Step 6: Unchanged records
    unchanged_records = df_tgt.join(
        changed_records.select("category_key"), "category_key", "leftanti"
    ).select(final_columns)

    # Step 7: Combine results
    final_df = unchanged_records.union(new_versions).union(old_versions).union(new_records)

    return final_df.select(final_columns)

# Apply SCD Type 2 changes
result_df = apply_scd_type2_changes(df_src, df_tgt)
display(result_df)

category_id,category_key,category_name,is_current,start_date,end_date
1,1,Audio,1,2025-01-16,null
2,2,TV and Video,1,2025-01-16,null
4,4,Cameras and camcorders,1,2025-01-16,null
5,5,Cell phones,1,2025-01-16,null
6,6,"Music, Movies and Audio Books",1,2025-01-16,null
7,7,Games and Toys,1,2025-01-16,null
8,8,Home Appliances,1,2025-01-16,null
10,3,NEW_Computers,1,2025-01-16,null
3,3,Computers,0,2025-01-16,2025-01-15


# Write results to table

In [0]:

result_df.write.format('delta') \
                  .mode('overwrite') \
                  .option('path', 'abfss://gold@contosoprojectstorage.dfs.core.windows.net/dim_category') \
                  .saveAsTable('contoso_catalog.gold.dim_category')